In [5]:
import numpy as np
import pandas as pd
import plotly.express as px
import sweetviz as sv
from funciones import datos, analisis

### **Datos**

In [78]:
base = pd.read_excel(r'..\DB\titanic.xlsx')

In [79]:
df_person = pd.read_excel(r'..\DB\titanic.xlsx', sheet_name='person')

In [80]:
df_country = pd.read_excel(r'..\DB\titanic.xlsx', sheet_name='Country')

In [81]:
base.shape

(1309, 13)

In [82]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   1309 non-null   int64  
 1   PassengerId  1309 non-null   int64  
 2   Survived     891 non-null    float64
 3   Pclass       1309 non-null   int64  
 4   Name         1309 non-null   object 
 5   Sex          1309 non-null   object 
 6   Age          1046 non-null   float64
 7   SibSp        1309 non-null   int64  
 8   Parch        1309 non-null   int64  
 9   Ticket       1309 non-null   object 
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
 12  TicketCost   1309 non-null   float64
dtypes: float64(3), int64(5), object(5)
memory usage: 133.1+ KB


### **Preparación de los Datos**

In [17]:
df_person = df_person.rename(columns={'PassengerID':'PassengerId'})

In [23]:
df_person.CountryOrigin.value_counts(dropna=False, normalize=True)

2    0.214231
4    0.212701
3    0.194338
1    0.190513
5    0.188217
Name: CountryOrigin, dtype: float64

In [25]:
df_country['CountryOrigin'] = df_country['id'].astype(str).str[-1].astype(int)

In [26]:
df_country.head()

,id,Country,ConversionRate,CountryOrigin
0,RUS01,Russia,79.78,1
1,GB-ENG02,England,0.77,2
2,ITA03,Italy,0.92,3
3,JPN04,Japan,128.38,4
4,FRA05,France,0.92,5


In [27]:
base.head()

,Unnamed: 0,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Cabin,Embarked,TicketCost
0,1,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,NaN,S,3851.4
1,2,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,C85,C,3465.0
2,3,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,NaN,S,27.6
3,4,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,C123,S,4140.0
4,5,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,NaN,S,2393.4


In [28]:
df_person.head()

,PassengerId,Name,CountryOrigin,Age,Revenue
0,1,"Braund, Mr. Owen Harris",4,22.0,1956.898317
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",2,38.0,3296.917137
2,3,"Heikkinen, Miss. Laina",5,26.0,2292.027450
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",5,35.0,3050.392669
4,5,"Allen, Mr. William Henry",1,35.0,3038.913629


In [29]:
base_union = base.merge(df_person[['PassengerId','CountryOrigin','Revenue']], how='left', on='PassengerId')\
                 .merge(df_country[['CountryOrigin','Country','ConversionRate']], how='left', on='CountryOrigin')

In [30]:
base.shape, base_union.shape

((1309, 13), (1309, 17))

In [35]:
base_union.head()

,Unnamed: 0,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Cabin,Embarked,TicketCost,CountryOrigin,Revenue,Country,ConversionRate,Revenue_euros,TicketCost_euros
0,1,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,NaN,S,3851.4,4.0,1956.898317,Japan,128.38,15.24,30.0
1,2,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,C85,C,3465.0,2.0,3296.917137,England,0.77,4281.71,4500.0
2,3,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,NaN,S,27.6,5.0,2292.027450,France,0.92,2491.33,30.0
3,4,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,C123,S,4140.0,5.0,3050.392669,France,0.92,3315.64,4500.0
4,5,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,NaN,S,2393.4,1.0,3038.913629,Russia,79.78,38.09,30.0


In [32]:
base_union['Revenue_euros'] = round(base_union['Revenue'] / base_union['ConversionRate'], 2)

In [41]:
base_union['TicketCost_euros'] = round(base_union['TicketCost'] / base_union['ConversionRate'])

### **Entendimiento de los Datos**

In [47]:
base_union.groupby('Pclass')[['TicketCost_euros']].describe(percentiles=[round(i, 2) for i in np.arange(0.1, 1, 0.1)]).T

Pclass                              1              2            3
TicketCost_euros count     323.000000     277.000000   707.000000
                 mean    39605.783282   13840.101083   289.086280
                 std    132548.602122   42668.853378   978.705367
                 min        27.000000       7.000000     0.000000
                 10%      3766.000000    1046.000000    25.000000
                 20%      4500.000000    1250.000000    30.000000
                 30%      4500.000000    1250.000000    30.000000
                 40%      4500.000000    1250.000000    30.000000
                 50%      4500.000000    1250.000000    30.000000
                 60%      4500.000000    1250.000000    30.000000
                 70%      4500.000000    1250.000000    30.000000
                 80%      4500.000000    1250.000000    30.000000
                 90%      5377.000000    1494.000000    36.000000
                 max    750273.000000  208409.000000  5002.000000

In [49]:
px.box(base_union, x='Pclass', y='TicketCost_euros')

In [50]:
base_union.columns

Index(['Unnamed: 0', 'PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age',
       'SibSp', 'Parch', 'Ticket', 'Cabin', 'Embarked', 'TicketCost',
       'CountryOrigin', 'Revenue', 'Country', 'ConversionRate',
       'Revenue_euros', 'TicketCost_euros'],
      dtype='object')

In [52]:
base_union.isnull().sum()

Unnamed: 0             0
PassengerId            0
Survived             418
Pclass                 0
Name                   0
Sex                    0
Age                  263
SibSp                  0
Parch                  0
Ticket                 0
Cabin               1014
Embarked               2
TicketCost             0
CountryOrigin          2
Revenue              265
Country                2
ConversionRate         2
Revenue_euros        265
TicketCost_euros       2
dtype: int64

In [53]:
base_union.Survived.value_counts(dropna=False, normalize=True)

0.0    0.419404
NaN    0.319328
1.0    0.261268
Name: Survived, dtype: float64

In [56]:
base_union['Ticket']

0                A/5 21171
1                 PC 17599
2         STON/O2. 3101282
3                   113803
4                   373450
               ...        
1304             A.5. 3236
1305              PC 17758
1306    SOTON/O.Q. 3101262
1307                359309
1308                  2668
Name: Ticket, Length: 1309, dtype: object

In [57]:
print(base_union.columns.tolist())

['Unnamed: 0', 'PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Cabin', 'Embarked', 'TicketCost', 'CountryOrigin', 'Revenue', 'Country', 'ConversionRate', 'Revenue_euros', 'TicketCost_euros']


In [58]:
campos = ['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Cabin'
         , 'Embarked', 'TicketCost', 'CountryOrigin', 'Revenue', 'Country', 'ConversionRate', 'Revenue_euros', 'TicketCost_euros']

### **Análisis de las Variables**

In [59]:
analyze_report = sv.analyze(base_union.query(" Survived in (0,1) ")[campos], target_feat='Survived')

Done! Use 'show' commands to display/save.   |██████████| [100%]   00:02 -> (00:00 left)


In [77]:
analyze_report.show_html('../Informes/Report_Variables.html')

Report ../Informes/Report_Variables.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [61]:
base_union_ft = base_union.query(" Survived in (0,1) ")

In [63]:
base_union_ft.isnull().sum()

Unnamed: 0            0
PassengerId           0
Survived              0
Pclass                0
Name                  0
Sex                   0
Age                 177
SibSp                 0
Parch                 0
Ticket                0
Cabin               687
Embarked              2
TicketCost            0
CountryOrigin         0
Revenue             177
Country               0
ConversionRate        0
Revenue_euros       177
TicketCost_euros      0
dtype: int64

In [64]:
base_union_ft[['Age','Revenue_euros']].describe(percentiles=[round(i, 2) for i in np.arange(0.1, 1, 0.1)])

,Age,Revenue_euros
count,714.000000,714.000000
mean,29.699118,1814.287591
std,14.526497,1876.657395
min,0.420000,1.170000
10%,14.000000,15.948000
20%,19.000000,23.120000
30%,22.000000,33.708000
40%,25.000000,62.406000
50%,28.000000,1738.360000
60%,31.800000,2319.976000


In [68]:
px.scatter(base_union_ft, x='Age', y='Revenue')

In [65]:
px.scatter(base_union_ft, x='Age', y='Revenue_euros')

In [67]:
analisis.correlacion(base_union_ft).query(" FirstVariable=='Age' ")

,FirstVariable,SecondVariable,Correlation
1,Age,Revenue,0.9999
7,Age,Revenue_euros,0.4555
11,Age,TicketCost_euros,0.3714
18,Age,SibSp,0.3082
20,Age,Parch,0.1891
22,Age,TicketCost,0.1836
67,Age,CountryOrigin,0.0053
68,Age,ConversionRate,0.0052
131,Age,Pclass,0.0000
132,Age,Survived,0.0000


In [71]:
base_union_ft['Survived'] = base_union_ft['Survived'].apply(lambda x: 1 if x==0 else 0) 

C:\Users\Acer\AppData\Local\Temp/ipykernel_8268/879106886.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [73]:
df_bivariado = analisis.bivariado(base_union_ft, campos, 'Survived')

In [74]:
df_bivariado.head()

,Variable,Value,All,Good,Bad,Share,Bad Rate,Distribution Good,Distribution Bad,WoE,IV,Total IV
1,Survived,0,342.0,342.0,0.0,0.383838,0.000000,1.000000,0.000000,0.000000,0.000000,0.00000
0,Survived,1,549.0,0.0,549.0,0.616162,1.000000,0.000000,1.000000,0.000000,-0.000000,0.00000
1,Pclass,1,216.0,136.0,80.0,0.242424,0.370370,0.397661,0.145719,1.003916,0.252928,0.50095
2,Pclass,2,184.0,87.0,97.0,0.206510,0.527174,0.254386,0.176685,0.364485,0.028321,0.50095
0,Pclass,3,491.0,119.0,372.0,0.551066,0.757637,0.347953,0.677596,-0.666483,0.219701,0.50095


In [76]:
df_bivariado.groupby('Variable').agg({'Total IV':'max'}).sort_values(by='Total IV', ascending=False).head(10)

,Total IV
Variable,
Sex,1.341681
TicketCost,0.581210
Pclass,0.500950
TicketCost_euros,0.500950
Age,0.320639
SibSp,0.142434
Embarked,0.122375
Cabin,0.119972
Parch,0.115172
